### Load packages
First we need to load the python packages that are required to perform the analysis

In [ ]:
%matplotlib inline

import os
import rasterio
import numpy 
import pandas
import scipy.io # to load the matlab files

import matplotlib.pyplot as plt

from DamageScanner import DamageScanner # the tool to estimate the losses

data_path = os.path.join('..','Data')

### Set paths to data and load initial data

All the data is stored in the *Data* folder, so we are creating paths to load data from that folder. We will create a dictionary with the depth-damage information required to do the analysis.

In [ ]:
# function to load DDM data
def read_DDM_input(DDM_data_path):
    DDM_data = {}
    DDM_data['Curves_structure'] = pandas.read_excel(DDM_data_path,sheet_name='curves_structure').values
    DDM_data['Curves_content'] = pandas.read_excel(DDM_data_path,sheet_name='curves_content').values
    DDM_data['MaxDam_structure'] = pandas.read_excel(DDM_data_path,sheet_name='maxdam_structure').values
    DDM_data['MaxDam_content'] = pandas.read_excel(DDM_data_path,sheet_name='maxdam_content').values  
    DDM_data['Landuse_names'] = dict(pandas.read_excel(DDM_data_path,sheet_name='landuse_list').values)  
    
    return DDM_data

In [ ]:
inundation_map = os.path.join(data_path,'flood_map.tif') # the land-use map
landuse_map = os.path.join(data_path,'landuse.tif') # the land-use map
DDM_data = read_DDM_input(os.path.join(data_path,'DDM_data.xlsx'))

### Run the DamageScanner. We run it for both the structure and the content

In [ ]:
# run the DamageScanner for the building structures
damage_structure,damagebin_structure = DamageScanner(landuse_map,inundation_map,DDM_data['Curves_structure'],
                                                     DDM_data['MaxDam_structure'])

# run the DamageScanner for the content
damage_content,damagebin_content = DamageScanner(landuse_map,inundation_map,DDM_data['Curves_content'],
                                                     DDM_data['MaxDam_content'])
# summarize both
damage_total = damage_content+damage_structure
damagebin_total  = pandas.DataFrame(pandas.concat([pandas.DataFrame(damagebin_structure['damages']), pandas.DataFrame(damagebin_content['damages'])], axis=1).fillna(0).sum(axis=1),columns=['damages'])
damagebin_total = pandas.concat([damagebin_total,damagebin_structure[['area','avg_depth']]],axis=1)
damagebin_total.index = damagebin_total.index.astype(int)
damagebin_total.index = damagebin_total.index.map(DDM_data['Landuse_names'])

#print output
print('{} Euro damage for a 1/1000 flood event'.format(damage_total))

### Translate land-use classes to economic sectors
We will focus in the remainder of the assignment on the impact of 
the industrial sectors to the economy the Rotterdam area

In [ ]:
# get only the industrial damages
industries = damagebin_total.iloc[15:29,:]

In [ ]:
plt.figure(figsize=(20,10))
plt.xticks(rotation=90) # add configurations
plt.xlabel('Landuse') # add configurations
plt.ylabel('Damage') # add configurations

plt.yticks() # add configurations
plt.xticks() # add configurations

plt.title('a title') # add configurations

industries['damages'].plot(kind='bar') # add configurations such as color or lining

plt.tight_layout()
output_path = os.path.join(data_path,'..','output')
plt.savefig(os.path.join(output_path,'1000yeardamage.png'),dpi=300)

### Create a mapping dictionary, to translate the land use names to sector codes

In [ ]:
mapping_dict = {'Industrial area':'C1','Industrial building':'C1','Warehouses':'H',
                'Food industry':'C2','Chemical industry':'C4','Building materials':'F',
               'Metal products industry':'C5','Transport industry':'H','Waste industry':'C8','Tank storage':'H',
                'Container terminals':'H','Break bulk cargo':'H','Bulk terminals':'H','Other industry':'C8'}

In [ ]:
industries.index = industries.index.map(mapping_dict)

### And save the total damages to a csv file, that we can use in the next phase

In [ ]:
output_path = os.path.join(data_path,'..','output')
pandas.DataFrame(industries.groupby(level=0).sum()['damages']).to_csv(os.path.join(output_path,'damages_sector.csv'))